# Budget Status Report

***Goal***: This notebook should:
- [X] Pull all active budgets for a PI
- [X] For each one, show the revenue and expenditures for a given date period
- [X] For each one, show the actual balance vs. time and compare it to the ideal burn rate
- [X] Show promised funds vs actual disbursements (Promised funds may have to be entered anually)
- [ ] Show who has been and is being funded
- [ ] Forcast each grant into the future, using a plot
- [ ] Allow the PI to model the assignment of future funds to specific salaries, equipment, etc. to see how long they will last

# Initialization

In [ ]:
from datetime import datetime
import sys
sys.path.insert(0, "/home/jovyan/work/src")
import pyedw as edw
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.dates as mdates
import matplotlib.cbook as cbook

edw.kinit()

In [ ]:
# Notebook Variables

PIName = "Last, First"
PINetID = "uwnetid"
OrgName = "%LASTNAME%"

# PI Information

This section finds information about the PI. 

In [ ]:
# PI Org Code

df = edw.str_query(
    "SELECT TOP 1 * FROM sec.FinancialOrganization WHERE OrgName LIKE '%s'" % OrgName,
    "ODS", 
    debug=False)
OrgCode = int(df.OrgCode[0])

# Figure out a PI's Employee ID
PIInfo = edw.str_query(
    "SELECT TOP 1 * FROM sec.Person WHERE UWNetID = '%s'" % PINetID, 
    "ODS", 
    debug=False)
EId = int(PIInfo["EmployeeID"][0])

print(PIName)
print("   NetID:       ", PINetID)
print("   OrgCode:     ", OrgCode)
print("   Employee ID: ", EId)


# Budget List

In [ ]:
# Method 1: Use RAD_dimBudget. Does not obviously show start / end dates
budgets = edw.str_query(
    "SELECT * FROM sec.RAD_dimBudget WHERE PIId = " + str(EId), 
    "EDWPresentation", 
    debug=False)
budget_numbers = budgets[["ReportingBudgetNbr", "BudgetName"]].drop_duplicates().reset_index()
budget_numbers

In [ ]:
# Method 2: Use BudgetIndex
# Note, this does not list budgets that have end dates = 0, which seem to be everlasting budgets like RCR.
# Columns explained at: https://metadata.uw.edu/Catalog/ViewItem/Table/odshepps.budgetindex
budgets = edw.str_query(
    "SELECT * FROM sec.BudgetIndex WHERE OrgCode = %i" % OrgCode,
    "ODS", 
    debug=False)
budgets["TotalPeriodEndDate"] = budgets["TotalPeriodEndDate"].astype(int)
budgets = budgets[(budgets["TotalPeriodEndDate"] >= 20200000)].drop_duplicates(subset=['BudgetNbr'])
budgets[["BudgetNbr", "BudgetName", "TotalPeriodBeginDate", "TotalPeriodEndDate", "RevenueSourceDesc", "HomeDeptOrgName", "BudgetTypeDesc", "BudgetStatus"]]

# Balance vs Time

In [ ]:
# Table defined at https://metadata.uw.edu/Catalog/ViewItem/Table/odshepps.budgetactivitydetail

budget_info = budgets.iloc[0] 
budget_nbr = budget_info["BudgetNbr"]
award_total = 332500

award_start = budget_info["TotalPeriodBeginDate"]
award_start = datetime.strptime(award_start, '%Y%m%d') 

award_end = str(budget_info["TotalPeriodEndDate"])
award_end = datetime.strptime(award_end, '%Y%m%d')

activity = edw.str_query(
    """SELECT * FROM sec.BudgetActivityDetail 
                WHERE BudgetNbr = '%s'
                AND TranCode IN (30,32,35,36,44,50,52,60,61,62,65,70,73)
    """ % budget_nbr,
    "ODS", 
    debug=False)

activity = activity.sort_values(by="TranPostingDate")
activity["TranAmount"] = activity["TranAmount"].astype(float)

income = activity[
    (activity["TranCode"] == '32') | (activity["TranCode"] == '35')
].copy()

income["Balance"] = income["TranAmount"].astype(float).cumsum()

transactions = activity[
    (activity["TranCode"] != '32') & (activity["TranCode"] != '35')
].copy()

transactions["Balance"] = transactions["TranAmount"].astype(float).cumsum()


In [ ]:
years = mdates.YearLocator()   # every year
months = mdates.MonthLocator()  # every month
years_fmt = mdates.DateFormatter('%Y')

income_balances = np.array(income["Balance"].values)
income_dates = np.array([np.datetime64(d) for d in income["TranPostingDate"].values])

transaction_balances = np.array(transactions["Balance"].values)
transaction_dates = np.array([np.datetime64(d) for d in transactions["TranPostingDate"].values])

burn_dates = np.array([np.datetime64(d) for d in [ award_start, award_end ] ]) 
burn_amounts = [ 0, award_total ]

fig = plt.figure()
ax = fig.add_subplot(1,1,1) 
plt.step(income_dates, -income_balances, transaction_dates, transaction_balances)
plt.plot(burn_dates, burn_amounts, burn_dates, [award_total, award_total]);

# format the ticks
ax.xaxis.set_major_locator(years)
ax.xaxis.set_major_formatter(years_fmt)
ax.xaxis.set_minor_locator(months)

# round to nearest years
datemin = np.datetime64(award_start, 'Y')
datemax = np.datetime64(award_end, 'Y') + np.timedelta64(1, 'Y')
ax.set_xlim(datemin, datemax)

# format the coords message box
ax.format_xdata = mdates.DateFormatter('%Y-%m-%d')
ax.grid(True)

# rotates and right aligns the x labels, and moves the bottom of the
# axes up to make room for them
fig.autofmt_xdate()
fig.set_size_inches(18.5, 10.5)

ax.set_title(str(budget_nbr) + ": " + budget_info["BudgetName"])

plt.show()